<a href="https://colab.research.google.com/github/karnamohit/kranka_ucm/blob/master/tf1_test_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing all the useful libraries...

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Checking the version of TensorFlow, NumPy...

In [3]:
print('**********************************************')
print('TensorFlow version info:\t',tf.__version__)
#!pip show tensorflow
print('NumPy version info:\t \t',np.__version__)
print('**********************************************')

**********************************************
TensorFlow version info:	 1.14.0
NumPy version info:	 	 1.16.5
**********************************************


**Making, processing raw data**

Make up data (2x2, sequentially indexed matrices)...

In [0]:
series_data = np.random.rand(100,2,2)      # "time"-series data for a 2x2 evolving matrix with 100 time-steps, randomly initialized
tsteps = int(series_data.shape[0])         # total number of time-steps in the series
tsteps_train = int(tsteps/2)               # number of time-steps used for training
mat_row = int(series_data.shape[1])
mat_col = int(series_data.shape[2])

In [5]:
print(series_data[tsteps_train-2,:,:])
print(series_data[tsteps_train-1,:,:])
print(series_data[tsteps_train,:,:])

[[0.32151992 0.44540184]
 [0.56733968 0.55889824]]
[[0.26441279 0.86078299]
 [0.31433715 0.40940587]]
[[0.42089622 0.1248318 ]
 [0.58808604 0.73949695]]


Define a sequence (multiple consecutive 2x2 matrices) size, ```len_seq```, to be fed in as input (equivalent to the amount of memory in previous time-steps, in our case), and the amount of overlap, ```n_overlap``` between any two adjacent input sequences...

In [0]:
len_seq = 3
n_overlap = 2

if (n_overlap >= len_seq):
  raise Exception("n_overlap must be less than len_seq")

Build tensor of sequences, call it ```seq_series_data```...

In [9]:
n_seq = int((tsteps_train - (len_seq - n_overlap)) / (len_seq - n_overlap)) # number of sequences of size len_seq
#print(n_seq)

seq_series_data = np.zeros((n_seq,len_seq,mat_row,mat_col), dtype='object')

i = 0
#j = 0
k = 0
while (i <= n_seq):           # the training sequences start at time-step 1 (series_data[0] element) in this case
  try:
    for j in range(len_seq):
      seq_series_data[k,j,:,:] = series_data[i+j,:,:]
  except IndexError:
    break
  #print(seq_series_data[k,:,:,:])
  i += len_seq - n_overlap
  #j += 1
  k += 1
  #print(k,i)

print(type(seq_series_data))

<class 'numpy.ndarray'>


**Using ```tf.Data``` to manipulate shape of data fed to the model** 

Make a ```tf.placeholder``` object to represent input data, ```seq_series_data```...

In [0]:
inp_ph = tf.placeholder(seq_series_data.dtype, seq_series_data.shape)

Use ```tf.data``` for storing input data represented by the ```tf.placeholder``` object ```inp_ph```...

In [0]:
inp_slices = tf.data.Dataset.from_tensor_slices(inp_ph)
next_slice_item = inp_slices.make_initializable_iterator()

Run a ```tf.Session()``` (within the context manager) to verify sequential printing of the evolving 2x2 matrix...

In [26]:
with tf.Session() as sess:
  while True:
    try:
      sess.run(next_slice_item.initializer, feed_dict={inp_ph: seq_series_data})
      print(sess.run(next_slice_item.make_one_shot_iterator().get_next()))
    except tf.errors.OutOfRangeError:
      break

InternalError: ignored

Define batches of the ```input_slices``` (saves memory when building the computational graph)

In [0]:
input_batches = input_slices.batches(N,drop_remainder=True)

Set up placeholders for input, true output, predicted output...

In [0]:
x = tf.placeholder(tf.float)